In [1]:
# sourcery skip: avoid-global-variables

import os
from loguru import logger as log
import pandas as pd
from pandas.io.formats.style import Styler
import numpy as np
import skombo
from skombo.combo_calc import parse_combos_from_csv
from skombo.fd_ops import FD, FD_BOT_CSV_MANAGER, COLS
import seaborn as sns

# dict comprehension to get the column names, but with the first letter capitalized and underscores replaced with spaces


test_combo_csv_path = skombo.TEST_COMBO_CSVS[0]

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 500)
# Round to 2 decimal places
pd.options.display.float_format = "{:,.2f}".format


def make_look_good(styler: Styler):
    styler.set_caption("Frame Data")
    styler.format(precision=2, na_rep="-")
    styler.format(precision=0, na_rep="-", subset=["Damage", "Combo Damage", "Hits"])
    # background gradient for the frame data
    styler.background_gradient(cmap="Blues", axis=0)
    return styler


def color_code(col: pd.Series, pal: str):
    unique = col.loc[col.apply(pd.notna)].unique()
    palette = sns.color_palette(pal, len(unique))
    # dict of {unique value: colour}
    colour_map = {unique[i]: palette[i] for i in range(len(unique))}  # type: ignore
    # Seperate dict for text colour, invert the hex values for now
    text_colour_map = dict(colour_map)
    # r,g,b is a tuple of floats
    for k, v in text_colour_map.items():
        r, g, b = v
        luma = 0.2126 * r + 0.7152 * g + 0.0722 * b  # type: ignore
        text_colour_map[k] = "#ffffff" if luma < 0.5 else "#000000"

    # convert to hex
    colour_map = {
        k: sns.palettes.color_palette([v]).as_hex()[0] for k, v in colour_map.items()  # type: ignore
    }
    text_colour_map = {
        k: sns.palettes.color_palette([v]).as_hex()[0]  # type: ignore
        for k, v in text_colour_map.items()
    }

    def color_cell(val):
        return (
            f"background-color: {colour_map[val]} ; color: {text_colour_map[val]}"
            if pd.notna(val)
            else "background-color: grey"
        )

    return col.apply(color_cell)

🌟|INFO    | Constants initialized
🌟|INFO    | ABS_PATH: C:\Users\micah\repos\skugs\src\skombo
🌟|INFO    | MODULE_NAME: skombo
🌟|INFO    | Logger initialized


In [2]:
from skombo.combo_calc import flatten_combo_df

combos, combo_damage = parse_combos_from_csv(test_combo_csv_path, calc_damage=True)

combo_display_columns = {
    COLS.char: "Character",
    COLS.m_name: "Move Name",
    "num_hits": "Hits",
    "scaled_damage": "Damage",
    COLS.mod_scaling: "Scaling",
    "summed_damage": "Combo Damage",
}

🌟|INFO    | ========== Parsing combos from csv [C:\Users\micah\repos\skugs\tests\test_data\annie_test_combos.csv] ==========
🌟|INFO    | Parsing [8] combos from csv
🌟|INFO    | Parsing combo for [ANNIE] : ['2LK', '2MK', '2HK']
🌟|INFO    | Parsing combo for [ANNIE] : ['2LK', '2MK', '5HP', '214HP~P', '236PP']
🌟|INFO    | Parsing combo for [ANNIE] : ['2LK', '2MK', '5HK', 'jHP', 'jHK', '236PP']
🌟|INFO    | Parsing combo for [ANNIE] : ['2LK', '2MK', '5HK', 'jLP', 'jLK', 'jMP', 'j236K~K', 'ADC', 'jMP', 'restand', '5LK', '2MP', '5MKx2', '5HP', '214HP~P', '236PP']
🌟|INFO    | Parsing combo for [ANNIE] : ['2LK', '2MK', '5HPx2', 'kara', 'j236HK', 'ADC', 'jHP', '5HK', 'jMK', 'j236HK~LK', '2MP', '5MKx2', '5HK', 'jMP', 'ADC', 'jLP', 'jMP', '5LK', '2MP', '5MKx2', '2HP', '623HP', '236PP']
🌟|INFO    | Parsing combo for [ANNIE] : ['2LK', '2MK', '5HPx2', 'kara', 'j236HK', 'ADC', 'jHP', '5HK', 'jLK', 'jMK', 'ADC', 'jMP', '5LPx2', '5MKx2', '5HK', 'jLP', 'jLK', 'jMP', '5LK', '2MP', '5MKx2', '2HP', 'OTG', '

In [3]:
for combo in combos:
    # LOG.info(f"Combo: \n{tabulate(combo, headers='keys', tablefmt='psql')}")  # type: ignore

    flat_combo = flatten_combo_df(combo)
    display_combo: pd.DataFrame = flat_combo[list(combo_display_columns.keys())].rename(
        combo_display_columns, axis=1
    )

    display(
        display_combo.style.pipe(make_look_good).apply(
            color_code, pal="pastel", subset=["Character"]  # type: ignore
        )
    )

,Character,Move Name,Hits,Damage,Scaling,Combo Damage
0,ANNIE,2LK,1,250,1.00,250
1,ANNIE,2MK,1,575,1.00,825
2,ANNIE,2HK,2,1487,1.00,2312


,Character,Move Name,Hits,Damage,Scaling,Combo Damage
0,ANNIE,2LK,1,250,1.00,250
1,ANNIE,2MK,1,575,1.00,825
2,ANNIE,5HP,1,700,1.00,1525
3,ANNIE,H NORTH KNUCKLE,1,568,0.88,2093
4,ANNIE,H NORTH KNUCKLE FOLLOWUP,1,689,0.77,2782
5,ANNIE,SAGAN BEAM,23,1633,0.67,4415


,Character,Move Name,Hits,Damage,Scaling,Combo Damage
0,ANNIE,2LK,1,250,1.00,250
1,ANNIE,2MK,1,575,1.00,825
2,ANNIE,5HK,1,950,1.00,1775
3,ANNIE,JHP,1,765,0.88,2540
4,ANNIE,JHK,2,1214,0.77,3754
5,ANNIE,SAGAN BEAM,23,1516,0.59,5270


,Character,Move Name,Hits,Damage,Scaling,Combo Damage
0,ANNIE,2LK,1,250,1.00,250
1,ANNIE,2MK,1,575,1.00,825
2,ANNIE,5HK,1,950,1.00,1775
3,ANNIE,JLP,1,218,0.88,1993
4,ANNIE,JLK,1,229,0.77,2222
5,ANNIE,JMP,4,553,0.67,2775
6,ANNIE,RE ENTRY,1,117,0.39,2892
7,ANNIE,RE ENTRY X2,1,137,0.34,3029
8,ANNIE,ADC,-,-,-,-
9,ANNIE,JMP,4,247,0.30,3276


,Character,Move Name,Hits,Damage,Scaling,Combo Damage
0,ANNIE,2LK,1,250,1.00,250
1,ANNIE,2MK,1,575,1.00,825
2,ANNIE,5HP,1,700,1.00,1525
3,ANNIE,5HP X2,-,-,-,-
4,ANNIE,KARA,-,-,-,-
5,ANNIE,RE ENTRY,1,262,0.88,1787
6,ANNIE,ADC,-,-,-,-
7,ANNIE,JHP,1,669,0.77,2456
8,ANNIE,5HK,1,636,0.67,3092
9,ANNIE,JMK,1,366,0.59,3458


,Character,Move Name,Hits,Damage,Scaling,Combo Damage
0,ANNIE,2LK,1,250,1.00,250
1,ANNIE,2MK,1,575,1.00,825
2,ANNIE,5HP,1,700,1.00,1525
3,ANNIE,5HP X2,-,-,-,-
4,ANNIE,KARA,-,-,-,-
5,ANNIE,RE ENTRY,1,262,0.88,1787
6,ANNIE,ADC,-,-,-,-
7,ANNIE,JHP,1,669,0.77,2456
8,ANNIE,5HK,1,636,0.67,3092
9,ANNIE,JLK,1,175,0.59,3267


,Character,Move Name,Hits,Damage,Scaling,Combo Damage
0,ANNIE,2LK,1,250,1.00,250
1,ANNIE,2MK,1,575,1.00,825
2,ANNIE,2HP,1,1200,1.00,2025
3,ANNIE,M CRESCENT CUT,1,481,0.88,2506
4,ANNIE,5MP,1,382,0.77,2888
5,ANNIE,5HP,1,468,0.67,3356
6,ANNIE,5HP X2,-,-,-,-
7,ANNIE,KARA,-,-,-,-
8,ANNIE,RE ENTRY,1,175,0.59,3531
9,ANNIE,AD,-,-,-,-


,Character,Move Name,Hits,Damage,Scaling,Combo Damage
0,ANNIE,2LK,1,250,1.00,250
1,ANNIE,2MK,1,575,1.00,825
2,ANNIE,5HP,1,700,1.00,1525
3,ANNIE,5HP X2,-,-,-,-
4,ANNIE,KARA,-,-,-,-
5,ANNIE,RE ENTRY,1,262,0.88,1787
6,ANNIE,ADC,-,-,-,-
7,ANNIE,JHP,1,669,0.77,2456
8,ANNIE,5HK,1,636,0.67,3092
9,ANNIE,JMK,1,366,0.59,3458
